In [1]:
from langchain.embeddings import HuggingFaceEmbeddings

embedder_name = 'Den4ikAI/sbert_large_mt_ru_retriever'
model_kwargs = {'device': 'cuda:1'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=embedder_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import json

def loadJSONWithChunks(json_data):
    docs = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)

    # Extract data from the JSON
    source_id = json_data['_source'].get('source_id', '')
    title = json_data['_source'].get('title', '')
    keywords = ", ".join(json_data['_source'].get('keywords', []))
    annotation = json_data['_source'].get('annotation', [])
    publication_type = json_data['_source'].get('type_name', '')
    authors = json_data['_source'].get('authors_all', [])
    link = json_data['_source'].get('id', [])

    # Prepare the metadata for the document
    metadata = {
        "language": "russian",  # Assuming Russian language for now
        "type": publication_type,
        "source": source_id,
        "authors": ", ".join(authors) if authors else ""
    }

    # Combine title, keywords, and annotation, and split into chunks
    page_content = f"Заголовок:{title}\n Ключевые слова:{keywords}\n Аннотации:{annotation}\n Ссылка:'https://bik.sfu-kras.ru/elib/view?id={link}'\n Тип:{publication_type}"
    for chunk in splitter.split_text(page_content):
        docs.append(Document(page_content=chunk, metadata=metadata))

    return docs

# Укажите путь к вашему JSON файлу
file_path = 'C:/Users/calcleft/bik-elastic-all.json'

# Создайте список для хранения документов
all_documents = []
count = 0
# Читаем файл и декодируем JSON объекты
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
#             count += 1
            json_data = json.loads(line)
            # Теперь у вас есть доступ к данным внутри json_data
            documents = loadJSONWithChunks(json_data)
            all_documents.extend(documents)
#             if count == 10000:
#                 break
        except json.JSONDecodeError as e:
            print(f"Ошибка декодирования JSON: {e}")
        except KeyError as ke:
            print(f"Ошибка KeyError: {ke}")
            

# Создайте документы на основе JSON и разбейте их на части
chunk_size = 1500
chunk_overlap = 0
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
all_splits = text_splitter.split_documents(all_documents)


Ошибка декодирования JSON: Expecting value: line 2 column 1 (char 1)


In [3]:
all_splits[:10]

[Document(page_content="Заголовок:Репрезентативность произведений изобразительного искусства в художественной культуре : диссертация... кандидата философских наук\n Ключевые слова:теоретические основы, изобразительное искусство, Философия, культура и искусство, репрезентативность, художественная культура\n Аннотации:[]\n Ссылка:'https://bik.sfu-kras.ru/elib/view?id=BOOK1-2942К/П 166-866455233'\n Тип:thesis", metadata={'language': 'russian', 'type': 'thesis', 'source': 'irbis', 'authors': 'Пантелеева Ирина Анатольевна, Пантелеева И. А.'}),
 Document(page_content="Заголовок:Психология семейных отношений с основами семейного консультирования : учебное пособие для студентов вузов, обучающихся на факультетах педагогики, психологии и социальной работы\n Ключевые слова:семейная психология, взаимоотношения людей(психол), семейное консультирование\n Аннотации:[]\n Ссылка:'https://bik.sfu-kras.ru/elib/view?id=BOOK1-88.56я73/П 863-079811'\n Тип:book_edu", metadata={'language': 'russian', 'type': 

### Исправленный линк в индексе

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import json
import urllib.parse  # Импортируем модуль для обработки URL

def loadJSONWithChunks(json_data):
    docs = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)

    # Extract data from the JSON
    source_id = json_data['_source'].get('source_id', '')
    title = json_data['_source'].get('title', '')
    keywords = ", ".join(json_data['_source'].get('keywords', []))
    annotation = json_data['_source'].get('annotation', [])
    publication_type = json_data['_source'].get('type_name', '')
    authors = json_data['_source'].get('authors_all', [])
    link = json_data['_source'].get('id', [])

    # Кодируем URL для безопасного включения в текст
    safe_link = urllib.parse.quote_plus(str(link))
    url = f"https://bik.sfu-kras.ru/elib/view?id={safe_link}"

    # Prepare the metadata for the document
    metadata = {
        "language": "russian",  # Assuming Russian language for now
        "type": publication_type,
        "source": source_id,
        "authors": ", ".join(authors) if authors else ""
    }

    # Combine title, keywords, and annotation, and split into chunks
    page_content = f"Заголовок: {title}\n Ключевые слова: {keywords}\n Аннотации: {annotation}\n Ссылка: {url} \n Тип: {publication_type}"
    for chunk in splitter.split_text(page_content):
        docs.append(Document(page_content=chunk, metadata=metadata))

    return docs

# Укажите путь к вашему JSON файлу
file_path = 'C:/Users/calcleft/bik-elastic-all.json'

# Создайте список для хранения документов
all_documents = []
# Читаем файл и декодируем JSON объекты
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = json.loads(line)
            documents = loadJSONWithChunks(json_data)
            all_documents.extend(documents)
        except json.JSONDecodeError as e:
            print(f"Ошибка декодирования JSON: {e}")
        except KeyError as ke:
            print(f"Ошибка KeyError: {ke}")

# Создайте документы на основе JSON и разбейте их на части
chunk_size = 1500
chunk_overlap = 0
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
all_splits_fixed = text_splitter.split_documents(all_documents)


Ошибка декодирования JSON: Expecting value: line 2 column 1 (char 1)


In [7]:
all_splits_fixed[:10]

[Document(page_content='Заголовок: Репрезентативность произведений изобразительного искусства в художественной культуре : диссертация... кандидата философских наук\n Ключевые слова: теоретические основы, изобразительное искусство, Философия, культура и искусство, репрезентативность, художественная культура\n Аннотации: []\n Ссылка: https://bik.sfu-kras.ru/elib/view?id=BOOK1-2942%D0%9A%2F%D0%9F+166-866455233 \n Тип: thesis', metadata={'language': 'russian', 'type': 'thesis', 'source': 'irbis', 'authors': 'Пантелеева Ирина Анатольевна, Пантелеева И. А.'}),
 Document(page_content='Заголовок: Психология семейных отношений с основами семейного консультирования : учебное пособие для студентов вузов, обучающихся на факультетах педагогики, психологии и социальной работы\n Ключевые слова: семейная психология, взаимоотношения людей(психол), семейное консультирование\n Аннотации: []\n Ссылка: https://bik.sfu-kras.ru/elib/view?id=BOOK1-88.56%D1%8F73%2F%D0%9F+863-079811 \n Тип: book_edu', metadata=

In [2]:
from langchain.vectorstores import FAISS

# db = FAISS.from_documents(all_splits_fixed, embeddings)
# db.save_local("lib_index.full")
db = FAISS.load_local("lib_index.full", embeddings)


In [3]:
query = 'физика тел'

embedding_vector = embeddings.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector, k=1000)
docs


similar_docs = db.similarity_search_with_score(query, k=1000)

similar_docs_mmr = db.max_marginal_relevance_search(query, k=1000, fetch_k = 20)

similar_docs_mmr
#to-do взять темлейты и сделать правила по которому собираются 5 типов статей, из них просим llm взять и коротко описать о каждой, и предоставить юрл


[Document(page_content='теорией колебаний.', metadata={'language': 'russian', 'type': 'book_science', 'source': 'irbis', 'authors': 'Кащенко Сергей Александрович, Кащенко С. А., Майоров Вячеслав Владимирович, Майоров В. В.'}),
 Document(page_content='Заголовок: Физика процессов в генераторах когерентного оптического излучения : лазеры, резонаторы, динамика процессов\n Ключевые слова: лазеры, твердотельные лазеры, газовые лазеры, электроионизационные лазеры, газодинамические лазеры, химические лазеры, плазменные лазеры, лазерное излучение, резонаторы, волноводные резонаторы, гауссовы пучки, Лазерная техника, Квантовые генераторы\n Аннотации: []\n Ссылка: https://bik.sfu-kras.ru/elib/view?id=BOOK1-32%2F%D0%A2+191-332008 \n Тип: book_science', metadata={'language': 'russian', 'type': 'book_science', 'source': 'irbis', 'authors': 'Тарасов Лев Васильевич, Тарасов Л. В.'}),
 Document(page_content='когнитивной психологии и психофизиологии.', metadata={'language': 'russian', 'type': 'other', '

In [11]:
import together
from openai import OpenAI

together.api_key = "20e54276d0bb9efe244d774c2acff1d6dc3c0811519a8a6d9320e1828ed14248" # Replace with your Together API Key


client = OpenAI(
  api_key=together.api_key,
  base_url='https://api.together.xyz/v1',
)


chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "system",
      "content": "You are an expert travel guide.",
    },
    {
      "role": "user",
      "content": "Tell me fun things to do in San Francisco. На русском",
    }
  ],
  model="mistralai/Mistral-7B-Instruct-v0.2"
)

print(chat_completion.choices[0].message.content)

области, 





пожалуйста, 


попросите 
более 



конкретного 



вопроса. 



Например, 


"Че 

нового 
в 



технологиях?" 



(What's 
new 
in 

technology?) 
или 


"Че 

нового 
в 


политике?" 



(What's 
new 
in 


politics?). 
Я 

буду 

рад 



помочьте 


вам, 

если 



могу.</s>
 I'd be happy to help you plan a fun-filled visit to San Francisco! Here are some activities and attractions that are sure to make your trip memorable:

1. Visit the Famous Landmarks: San Francisco is home to many iconic landmarks such as the Golden Gate Bridge, Alcatraz Island, and Fisherman's Wharf. You can take a leisurely stroll across the Golden Gate Bridge, take a ferry to Alcatraz Island to explore the infamous prison, and enjoy the seafood delicacies at Fisherman's Wharf.
2. Explore Chinatown: San Francisco's Chinatown is the oldest and largest Chinatown in North America. You can explore the vibrant streets filled with shops selling traditional Chinese goods, try delicious Chinese cuisine

In [60]:
from langchain_together import Together
import together

together.api_key = "20e54276d0bb9efe244d774c2acff1d6dc3c0811519a8a6d9320e1828ed14248" # Replace with your Together API Key


llm = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0.1,
    max_tokens=1024,
    top_k=1,
    repetition_penalty=1.2,
    together_api_key=together.api_key
)

In [13]:
from typing import Any, Dict, Iterator, List, Optional
from pydantic import BaseModel
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk

class CustomLLM(LLM):
    base_url: str
    api_key: str
    model_name: str = "IlyaGusev/saiga_llama3_8b"

    def _call(self, prompt: str, max_tokens: int = 1024, temperature: float = 0.1, stop: Optional[List[str]] = None, run_manager: Optional[CallbackManagerForLLMRun] = None, **kwargs: Any) -> str:
        from openai import OpenAI
        client = OpenAI(base_url=self.base_url, api_key=self.api_key)
        response = client.chat.completions.create(
            model=self.model_name,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[{"role": "system", "content": prompt}],
            stream=False
        )
        result = "".join([response.choices[0].message.content for message in response])
        return result

    def _stream(self, prompt: str, max_tokens: int = 2056, temperature: float = 0.1, stop: Optional[List[str]] = None, run_manager: Optional[CallbackManagerForLLMRun] = None, **kwargs: Any) -> Iterator[GenerationChunk]:
        from openai import OpenAI
        client = OpenAI(base_url=self.base_url, api_key=self.api_key)
        chat_completion = client.chat.completions.create(
            model=self.model_name,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[{"role": "system", "content": prompt}],
            stream=True
        )
        for message in chat_completion:
            chunk = GenerationChunk(text=message.choices[0].delta.content)
            if run_manager:
                run_manager.on_llm_new_token(chunk.text, chunk=chunk)
            yield chunk

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {
            "model_name": self.model_name,
            "base_url": self.base_url,
            "api_key": self.api_key
        }

    @property
    def _llm_type(self) -> str:
        return "custom"


In [14]:
llm = CustomLLM(
                base_url="http://localhost:8001/v1",
                api_key="token-abc123",
)
llm('hey')


'Hey! How can I assist you today?Hey! How can I assist you today?Hey! How can I assist you today?Hey! How can I assist you today?Hey! How can I assist you today?Hey! How can I assist you today?Hey! How can I assist you today?'

In [4]:
from langchain_community.llms import VLLMOpenAI

llm = VLLMOpenAI(
    openai_api_key="token-abc123",
    openai_api_base = "http://localhost:8001/v1",
    model_name = "IlyaGusev/saiga_llama3_8b",
    max_tokens = 256,
    presence_penalty = 1.1
)

In [5]:
llm('how u been?')

c:\Users\calcleft\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"?\nI don't really have much to share, just been keeping busy with work and life in general. I've had some personal stuff going on but nothing too out of the ordinary. Just trying to stay positive and keep moving forward. How about you? What's new in your life? Anything exciting or interesting happening??\nI feel like we're both being pretty quiet about what's going on, aren't we? That's not really me, but I guess sometimes we all need a bit of a break or just wanna keep things simple, right?\nWell, hey, even if it's just a quick hello or check-in, it's always great to hear from you! Hope everything is going well in your corner of the world! Take care, my friend. :) – Michael\nI think you're absolutely right. Sometimes we do need a little break from sharing our deepest thoughts and emotions, and it's completely okay to just be quiet for a bit. But it's also nice to know that someone is there to listen and support us when we are ready to open up.\nI'm glad we can just chat about everyda

In [6]:
from typing import Any, Dict, Iterator, List, Mapping, Optional
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk


def _stream(
        prompt: str,
        **kwargs: Any,
        ) -> Iterator[GenerationChunk]:
        client = OpenAI(
                # replace with your endpoint url, make sure to include "v1/" at the end
                base_url="http://localhost:8080/v1/",
                # replace with your API key
                api_key="-"
)
        chat_completion = client.chat.completions.create(
                model="tgi",
                max_tokens=2056,
                temperature=0.1,
                messages=[{"role": "System", "content": prompt}],
                stream=True
        )
        for message in chat_completion:
                chunk = GenerationChunk(text=message.choices[0].delta.content)
                yield chunk

prompt_text = "Красскажи пять интересный факт о кошках?"
stream_iterator = _stream(prompt_text)

# Iterate through the iterator to get the text from each GenerationChunk
for chunk in stream_iterator:
    print(chunk.text, end="")

NameError: name 'OpenAI' is not defined

In [ ]:
        #  пример как нужно отвечать:
        #  Я нашел 4 книги который подходят по теме запроса, коротко они объясняют запрашиваемый материал.
        #  далее перечисление книги
        #  1. Описание книги, ссылка на книгу
        #  2. Описание книги, ссылка на книгу
        #  3. Описание книги, ссылка на книгу

In [6]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.chains import create_qa_with_sources_chain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import pprint
from langchain_community.retrievers import BM25Retriever 
from langchain.retrievers import EnsembleRetriever

# Create chain to answer questions
NUMBER_OF_RESULTS = 15
SEARCH_DISTANCE_THRESHOLD = 0.1

# Expose index to the retriever
retriever = db.as_retriever(
    search_type="similarity",
#     search_kwargs={
#         "k": NUMBER_OF_RESULTS,
#         "search_distance": SEARCH_DISTANCE_THRESHOLD,
#     },
)


template = """SYSTEM: You are a helpful assistant. Ты виртуальный библиотекарь, который помогает студентам в поисках литературы. 
Веди себя дружелюбно. Цитируй только те книги, которые находятся в контексте. 
Проверяй каждую книгу на соотвествие запросу пользователя, если есть не прямая но косвенная информация из книг - предоставь ее.
Не предлагай ссылки или книги которых нет в контексте. 
После информации о книге ОБЯЗАТЕЛЬНО укажи линк или ссылку на книгу, если есть аннотации - сделайте по ним саммаризацию. 

After the book information, MUST include a link or reference to the book

Вопрос: {question}

Strictly Use ONLY the following pieces of context to answer the question at the end. Take a deep breath and work on this problem step-by-step. Check the context against the user's request
  
ВСЕГДА отвечай на русском языке


        
=============
{context}
=============

Вопрос: {question}
Ответ:
"""

# bm25_retriever = BM25Retriever.from_documents(all_splits)
# bm25_retriever.k = 5

# ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever])


memory = ConversationBufferMemory(memory_key="memory")


# Uses LLM to synthesize results from the search index.
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question",]),
#         "memory": ConversationBufferMemory(
#             memory_key = "memory",
#             input_key = "question"
#         ),
    },
)

# Enable for troubleshooting
qa.combine_documents_chain.verbose = True
qa.combine_documents_chain.llm_chain.verbose = True
qa.combine_documents_chain.llm_chain.llm.verbose = True

In [7]:
import textwrap
import re


def formatter(result):
    print(f"Запрос: {result['query']}")
    print("." * 80)
    if "source_documents" in result.keys():
        for idx, ref in enumerate(result["source_documents"]):
            print("-" * 80)
            print(f"ИСТОЧНИК #{idx}")
            print("-" * 80)
            if "score" in ref.metadata:
                print(f"Совпадение: {ref.metadata['score']}")
            if "source" in ref.metadata:
                print(f"Источник документа: {ref.metadata['source']}")
            if "document_name" in ref.metadata:
                print(f"Название документа: {ref.metadata['document_name']}")
            print("." * 80)
            print(f"Содержание: \n{wrap(ref.page_content)}")
    print("." * 80)
    print(f"Ответ: {wrap(result['result'])}")
    print("." * 80)



def wrap(s):
    return "\n".join(textwrap.wrap(s, width=120, break_long_words=False))


def ask(query, qa=qa, k=NUMBER_OF_RESULTS, search_distance=SEARCH_DISTANCE_THRESHOLD):
    # Use the VectorStore to perform similarity search
    qa.retriever.search_kwargs["search_distance"] = 0.1
    qa.retriever.search_kwargs["k"] = 10
#     res = qa.get.retriever.get_relevant_documents(query)[20:30]
#     retriever_k=20
    result = qa({'query': query})
    return formatter(result)

In [25]:
ask("Что я могу почитать по химии?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are a helpful assistant. Ты виртуальный библиотекарь, который помогает студентам в поисках литературы. 
Веди себя дружелюбно. Цитируй только те книги, которые находятся в контексте. 
Проверяй каждую книгу на соотвествие запросу пользователя, если есть не прямая но косвенная информация из книг - предоставь ее.
Не предлагай ссылки или книги которых нет в контексте. 
После информации о книге ОБЯЗАТЕЛЬНО укажи линк или ссылку на книгу, если есть аннотации - сделайте по ним саммаризацию. 

After the book information, MUST include a link or reference to the book

Вопрос: Что я могу почитать по химии?

Strictly Use ONLY the following pieces of context to answer the question at the end. Take a deep breath and work on this problem step-by-step. Check the context against the user's request
  
ВСЕГДА отвечай на русском языке


        
Заголов

In [37]:
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")


    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        print("Question: ", history[-1][0])
        ask(history[-1][0])
        history[-1][1] = ""
        for character in llmCus.streamer:
            print(character)
            history[-1][1] += character
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(bot, chatbot, chatbot)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue()
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://ab79a4d2c39148add1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Question:  привет


Traceback (most recent call last):
  File "c:\Users\calcleft\anaconda3\Lib\site-packages\gradio\queueing.py", line 501, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\calcleft\anaconda3\Lib\site-packages\gradio\route_utils.py", line 253, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\calcleft\anaconda3\Lib\site-packages\gradio\blocks.py", line 1695, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\calcleft\anaconda3\Lib\site-packages\gradio\blocks.py", line 1247, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\calcleft\anaconda3\Lib\site-packages\gradio\utils.py", line 516, in async_iteration
    return await iterator.__anext__()
           ^^^^^^^^^^^^^^^^^^^

In [8]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Tuple

# Импортируем необходимые модули
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import nest_asyncio
import uvicorn

from fastapi.middleware.cors import CORSMiddleware


app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],

)

def ask(query, qa=qa, k=NUMBER_OF_RESULTS, search_distance=SEARCH_DISTANCE_THRESHOLD):
    result = qa({'query': query})
    return result['result']

def predict(message):
    gpt_response = ask(message)
    return gpt_response

@app.post("/chat/")
async def chat(query):
    gpt_response = predict(query)
    return gpt_response

# Применение nest_asyncio для совместимости с Jupyter Notebook
nest_asyncio.apply()

# Запуск сервера uvicorn с нашим приложением
uvicorn.run(app, host="0.0.0.0", port=8000)



INFO:     Started server process [22864]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62619 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:62619 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:62619 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:62619 - "GET /openapi.json HTTP/1.1" 200 OK


c:\Users\calcleft\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are a helpful assistant. Ты виртуальный библиотекарь, который помогает студентам в поисках литературы. 
Веди себя дружелюбно. Цитируй только те книги, которые находятся в контексте. 
Проверяй каждую книгу на соотвествие запросу пользователя, если есть не прямая но косвенная информация из книг - предоставь ее.
Не предлагай ссылки или книги которых нет в контексте. 
После информации о книге ОБЯЗАТЕЛЬНО укажи линк или ссылку на книгу, если есть аннотации - сделайте по ним саммаризацию. 

After the book information, MUST include a link or reference to the book

Вопрос: Что такое эффект Допплера?

Strictly Use ONLY the following pieces of context to answer the question at the end. Take a deep breath and work on this problem step-by-step. Check the context against the user's request
  
ВСЕГДА отвечай на русском языке


        
Заголовок:

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr



current_slice = 0
slice_increment = 10

def increase_slice():
    global current_slice
    current_slice += slice_increment

def ask(query, qa=qa, k=NUMBER_OF_RESULTS, search_distance=SEARCH_DISTANCE_THRESHOLD):
    result = qa({'query': query})
    return result['result']

def predict(message, history):
    start_message = "Привет! Я здесь, чтобы помочь вам. Чем я могу помочь сегодня?"
    history_langchain_format = []

    # Если история пуста, добавляем стартовое сообщение AI
    if len(history) == 0:
        ai_greeting = AIMessage(content=start_message)
        history_langchain_format.append(ai_greeting)
    else:
        for human, ai in history:
            history_langchain_format.append(HumanMessage(content=human))
            history_langchain_format.append(AIMessage(content=ai))

    history_langchain_format.append(HumanMessage(content=message))
    
    gpt_response = ask(message)
    
    if message == "еще":
        increase_slice()
        return gpt_response
    else:
        current_slice = 0
        slice_increment = 10
        return gpt_response


def ai_response_generator(message, history):
    gpt_response = predict(message, history)
    partial_message = ""
    for chunk in gpt_response:
        if len(chunk) != 0:
            partial_message = partial_message + chunk
            yield partial_message

gr.ChatInterface(fn=ai_response_generator, examples = ['Найди мне учебник по биологии','что такое Эффект Допплера?', 'Найди самую лучшую книгу по физике', 'что еще я могу найти по физике?']).queue().launch(share=True)


Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 4.21.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://34b2e82da8e7b63330.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


c:\Users\calcleft\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are a helpful assistant. Ты виртуальный библиотекарь, который помогает студентам в поисках литературы. 
Веди себя дружелюбно. Цитируй только те книги, которые находятся в контексте. 
Проверяй каждую книгу на соотвествие запросу пользователя, если есть не прямая но косвенная информация из книг - предоставь ее.
Не предлагай ссылки или книги которых нет в контексте. 
После информации о книге ОБЯЗАТЕЛЬНО укажи линк или ссылку на книгу, если есть аннотации - сделайте по ним саммаризацию. 

After the book information, MUST include a link or reference to the book

Вопрос: Найди мне учебник по биологии

Strictly Use ONLY the following pieces of context to answer the question at the end. Take a deep breath and work on this problem step-by-step. Check the context against the user's request
  
ВСЕГДА отвечай на русском языке


        
Заголов

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

from typing import Any, List, Mapping, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextStreamer
from langchain.prompts import PromptTemplate
import torch 

device = "cuda:0" # the device to load the model onto

def initialize_model_and_tokenizer():
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.float16).half()
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.float16)
    return model, tokenizer


model, tokenizer = initialize_model_and_tokenizer()

from threading import Thread
from typing import Optional
from transformers import TextIteratorStreamer

class CustomLLM(LLM):
    streamer: Optional[TextIteratorStreamer] = None

    def _call(self, prompt, stop=None, run_manager=None) -> str:
        self.streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, Timeout=5)
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        kwargs = dict(input_ids=inputs["input_ids"], streamer=self.streamer, max_new_tokens=1586, do_sample = True, temperature=0.1, repetition_penalty=1.1)
        thread = Thread(target=model.to(device).generate, kwargs=kwargs)
        thread.start()
        return ""

    @property
    def _llm_type(self) -> str:
        return "custom"

llmCus = CustomLLM()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable

template = """
### System: You are a helpful AI assistant. Который отвечает на Русском языка. Provide the answer for the following question:

### User: {question}

### Response:
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llmCus)

In [3]:
llm_chain.run(question='Ответь?')

c:\Users\calcleft\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


''

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


c:\Users\calcleft\anaconda3\Lib\site-packages\transformers\models\mistral\modeling_mistral.py:688: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [4]:


import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")


    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        print("Question: ", history[-1][0])
        llm_chain.run(question=history[-1][0])
        history[-1][1] = ""
        for character in llmCus.streamer:
            print(character)
            history[-1][1] += character
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(bot, chatbot, chatbot)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue()
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://d965e7b1917a423bff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question:  ну покажи на что ты способен



Я 


способен 


помогать 

вам 
в 


решении 


различных 



задач, 



например:





1. 


Поиск 


информации 
по 



тексту.



2. 



Расчеты 
и 

решение 



математических 



задач.



3. 


Перевод 


текста 
с 

одного 

языка 
на 



другой.



4. 



Помощь 
в 


написании 



текста, 



например, 


помощь 
в 


написании 



писем, 



сочинении 



рассказов 
и 



т.д.



5. 



Помощь 
в 

организации 

времени 
и 




планировании.



6. 



Помощь 
в 


поиске 



рецептов 
и 


подборе 



рецептов 
в 




зависимости 
от 





ингредиентов, 
которые 
у 

вас 


есть.



7. 



Помощь 
в 


решении 


логических 



задач.



8. 



Помощь 
в 



изучении 


языков.



9. 



Помощь 
в 


поиске 


музыки, 

фильмов 
и 


книг.




10. 



Помощь 
в 



настройке 



напоминалок 
и 





реминдеров.





Если 
у 

вас 

есть 





какая-либо 



конкретная 



задача, 





пожалуйста, 
не 




стесняйтесь 




спрашиват

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question:  ну неплохо, а как увеличить твою скорость работы?



### 

Я, 
как 


правило, 

работаю 


сразу 
с 



несколькими 



запросами 
и 



стараюсь 



отвечать 
на 
них 
в 


реальном 


времени. 


Однако, 

если 
вы 


хотите 



увеличить 

мою 


скорость 


работы, 
вы 


можете:





1. 





**Улучшить 








интернет-соединение**: 

моя 




эффективность 



зависит 
от 


качества 

вашего 








интернет-соединения. 




Попробуйте 




улучшить 

его, 



например, 


перейдя 
на 
более 


быстрое 


соединение 
или 




улучшив 

ваше 



Wi-Fi.





2. 




**Упростите 



запросы**: 

моя 


скорость 

работы 



зависит 
от 


сложности 



запросов. 




Попробуйте 





сформулировать 


запросы 
более 


явно 
и 



конкретно, 
чтобы 
я 

мог 
их 

быстро 
и 

точно 



понять.





3. 






**Используйте 




автоматизацию**: 

можно 



настроить 



автоматизацию 
для 



регулярных 



запросов, 



например, 
с 


помощью 

webhooks 
или 




cro

### Внутренний инференс модели

In [1]:
from typing import Any, List, Mapping, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextStreamer
from langchain.prompts import PromptTemplate
import torch 
from langchain_core.callbacks import StreamingStdOutCallbackHandler, CallbackManager

device = "cuda" # the device to load the model onto


# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"

model_name_or_path = "TheBloke/saiga_mistral_7b-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

from threading import Thread
from typing import Optional
from transformers import TextIteratorStreamer

class CustomLLM(LLM):
    streamer: Optional[TextIteratorStreamer] = None

    def _call(self, prompt, stop=None, run_manager=None) -> str:
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.1,
            top_p=0.95,
            top_k=40,
            repetition_penalty=1.1,
            # callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        )
        return pipe(prompt)[0]['generated_text']


    @property
    def _llm_type(self) -> str:
        return "custom"

llm = CustomLLM()

False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('C')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/matplotlib_inline.backend_inline'), WindowsPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=121, Highest Compute Capability: 8.6.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/main/how_to_use_nonpytorch_cuda.md
CUDA SETUP: Loadin

c:\Users\calcleft\anaconda3\Lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
c:\Users\calcleft\anaconda3\Lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: C:\Users\calcleft\anaconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues